<a href="https://colab.research.google.com/github/hyper-delta/flu-vaccine/blob/main/IIT_Guwahati_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score


train_labels = pd.read_csv('/content/training_set_labels.csv')
train_features = pd.read_csv('/content/training_set_features.csv')
test_features = pd.read_csv('/content/test_set_features.csv')
submission_format = pd.read_csv('/content/submission_format.csv')


train_data = train_features.merge(train_labels, on='respondent_id')


features = train_data.drop(columns=['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'])
targets = train_data[['xyz_vaccine', 'seasonal_vaccine']]


categorical_features = features.select_dtypes(include=['object']).columns
numerical_features = features.select_dtypes(exclude=['object']).columns


numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))


pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])


X_train, X_val, y_train, y_val = train_test_split(features, targets, test_size=0.2, random_state=42)


pipeline.fit(X_train, y_train)


y_pred_proba = pipeline.predict_proba(X_val)


roc_auc_xyz = roc_auc_score(y_val['xyz_vaccine'], y_pred_proba[0][:, 1])
roc_auc_seasonal = roc_auc_score(y_val['seasonal_vaccine'], y_pred_proba[1][:, 1])
mean_roc_auc = (roc_auc_xyz + roc_auc_seasonal) / 2

print(f'ROC AUC for xyz_vaccine: {roc_auc_xyz}')
print(f'ROC AUC for seasonal_vaccine: {roc_auc_seasonal}')
print(f'Mean ROC AUC: {mean_roc_auc}')


test_features_processed = test_features.drop(columns=['respondent_id'])
predictions = pipeline.predict_proba(test_features_processed)
submission = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': predictions[0][:, 1],
    'seasonal_vaccine': predictions[1][:, 1]
})

submission.to_csv('submission.csv', index=False)

ROC AUC for xyz_vaccine: 0.864173999277244
ROC AUC for seasonal_vaccine: 0.8570519011081396
Mean ROC AUC: 0.8606129501926918
